In [1]:
from statsmodels.tsa.stattools import adfuller, kpss
import pandas as pd
from scipy.signal import hilbert
import numpy as np
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose
import matplotlib.pyplot as plt

In [2]:
def preenche_anterior(df, coluna):
    for nan in range(0, df[coluna].isna().sum()):
        nulo = df[df[coluna].isna()==True].index[0]
        anterior = df[coluna].loc[:nulo].dropna().index[-1]
        anterior = anterior.to_pydatetime()  #converte o objeto Timestamp em um Datetime
        anterior = anterior.strftime('%Y-%m-%d')
        nulo = nulo.to_pydatetime()
        nulo = nulo.strftime('%Y-%m-%d')
        df[coluna].loc[nulo] = df[coluna].loc[anterior]
    return df

In [3]:
def adf_test(timeseries):
    print("Results of Dickey-Fuller Test:")
    dftest = adfuller(timeseries, autolag="AIC")
    dfoutput = pd.Series(
        dftest[0:4],
        index=[
            "Test Statistic",
            "p-value",
            "#Lags Used",
            "Number of Observations Used",
        ],
    )
    for key, value in dftest[4].items():
        dfoutput[f"Critical Value ({key})"] = value
    print('Série estacionária se p-value < 0.05')
    print('====================================')
    print(dfoutput)

In [4]:
def kpss_test(x, h0_type='c'):
  indices = ['Estatística do teste', 'valor-p', '# de Lags']
  kpss_test = kpss(x, regression=h0_type, nlags="legacy")
  results = pd.Series(kpss_test[0:3], index=indices)
  for key, value in kpss_test[3].items():
    results[f'Critical Value ({key})'] = value
  print('Série estacionária se p-value > 0.05')
  print('====================================')
  return results

In [5]:
def fases_hilbert_transform(time_series):
  ang = np.angle(hilbert(time_series, axis=0))
  return np.mod(ang, 2*np.pi) #retorna ângulo entre 0 e 2pi

In [6]:
def plot_picos_acf_pacf(data):
    # Calcular a função de autocorrelação
    fig, ax = plt.subplots(2, 1, figsize=(12, 8))

    # Plotar a função de autocorrelação (ACF)
    plot_acf(data, lags=300, ax=ax[0])
    ax[0].set_title('Função de Autocorrelação')

    # Plotar a função de autocorrelação parcial (PACF)
    plot_pacf(data, lags=300, ax=ax[1])
    ax[1].set_title('Função de Autocorrelação Parcial')
    plt.show()

    # Calcular os valores da autocorrelação e os intervalos de confiança
    acf_values = ax[0].lines[0].get_ydata()
    conf_intervals = np.concatenate([path.vertices[:, 1] for path in ax[0].collections[1].get_paths()])

    # Identificar picos na ACF
    acf_peaks = np.where(acf_values > conf_intervals)[0]
    print("Picos na ACF:", acf_peaks)

    # Identificar picos na PACF
    pacf_values, _ = ax[1].lines[0].get_ydata(), ax[1].collections[1].get_segments()[0][:, 1]
    pacf_peaks = np.where(pacf_values > 0.1)[0]  # Ajuste o limiar conforme necessário
    print("Picos na PACF:", pacf_peaks)

In [7]:
# Função para remover tendência das séries
def remove_trend(serie, tipo):
  if tipo == 1:
    decomp = seasonal_decompose(serie, model='additive',period=252)
    result = decomp.seasonal + decomp.resid
  elif tipo == 2:
    decomp = seasonal_decompose(serie, model='multiplicative',period=252)
    result = decomp.seasonal * decomp.resid
  else:
    print("Tipo inválido")
  return result

In [8]:
def dinamica_fases_plot(df):
    N = len(df.columns)
    theta_t = fases_hilbert_transform(df)
    #theta_t = fases_hilbert_transform(retornos_df)
    r = np.sqrt( np.sum(np.cos(theta_t), axis=1)**2 + np.sum(np.sin(theta_t), axis=1)**2 ) / N

    f, ax = plt.subplots(3, 1, figsize=(14,8), sharex=True)
    ax[0].set(ylim=[0,2.1*np.pi], ylabel=r'$\theta_i$',title='Dinâmica das fases')
    for i in range(0,N):
        ax[0].plot(theta_t[:, i], label=r'$\theta_{%d}$' % (i+1))
        ax[0].legend()

    ax[1].set(ylabel=r'$\sin(\theta_i)$')
    for i in range(0,N):
        ax[1].plot(np.sin(theta_t[:, i]), label=r'$\sin(\theta_{%d})$' % (i+1))
        ax[1].legend()

    ax[2].set(ylim=[0.97,1.01],xlim=[0,len(r)],title='Sincronização de fases instantâneas',xlabel='Tempo',ylabel='Parâmetro de ordem')
    ax[2].plot(r)
    plt.tight_layout()
    plt.show()